In [1]:
import os
from google.colab import drive

In [2]:
drive.mount('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [3]:
root_dir = '/content/gdrive/My Drive/classification_project/'

In [4]:
os.chdir(root_dir)

In [5]:
print(os.getcwd())

/content/gdrive/My Drive/classification_project


In [6]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 9.6 MB/s 
     |████████████████████████████████| 3.3 MB 41.9 MB/s 
     |████████████████████████████████| 596 kB 62.3 MB/s 
     |████████████████████████████████| 895 kB 62.1 MB/s 
     |████████████████████████████████| 67 kB 7.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [7]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
import pandas as pd
import torch
import numpy as np
import tensorflow as tf
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm



In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased' , do_lower_case = True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [9]:
train = pd.read_csv('sentiment-train.csv')

In [10]:
test  = pd.read_csv('sentiment-test.csv')

In [11]:
train['short_review'] = train['review_body'].str.decode("utf-8")

In [12]:
test['short_review'] = test['review_body'].str.decode("utf-8")

In [13]:
train.columns

Index(['Unnamed: 0', 'marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'product_category', 'star_rating',
       'helpful_votes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date', 'review_label',
       'review_clean', 'review_tokenize', 'review_sentence_tokenize', 'labels',
       'short_review'],
      dtype='object')

In [14]:
test.columns

Index(['Unnamed: 0', 'marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'product_category', 'star_rating',
       'helpful_votes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date', 'review_label',
       'review_clean', 'review_tokenize', 'review_sentence_tokenize', 'labels',
       'short_review'],
      dtype='object')

In [15]:
train.head()

,Unnamed: 0,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,review_label,review_clean,review_tokenize,review_sentence_tokenize,labels,short_review
0,6919500,US,14844896,R16UP5FE6VKN3D,B00B5HEN7I,52588358,Lumii Ark Screen Protector Shield for BlackBer...,Wireless,5,0,0,0,1,Easy install,This screen cover is a very easy install unlik...,2013-05-18,positive,This screen cover easy install unlike get 3 pa...,"['This', 'screen', 'cover', 'easy', 'install',...",['This screen cover easy install unlike get 3 ...,0,NaN
1,6268938,US,19531791,R27WDZK00FPSMB,B008UQ14ZQ,184850555,Aimo Wireless Rubber Essentials Slim and Durab...,Wireless,5,3,3,0,1,Phone Protection,I just upgraded my cell phone and was concerne...,2013-10-02,positive,I upgraded cell phone concerned protecting it....,"['I', 'upgraded', 'cell', 'phone', 'concerned'...",['I upgraded cell phone concerned protecting i...,0,NaN
2,4842805,US,26617806,R2IDEL43VVG6K5,B00D5WNU8Y,955139685,"Galaxy S4 Case Leather, CellJoy® [Wallet Hybri...",Wireless,5,0,0,0,1,Celljoy Leather Wallet Credit Card,So loving this case it comes in handy when I w...,2014-06-25,positive,So loving case comes handy I want feminine app...,"['So', 'loving', 'case', 'comes', 'handy', 'I'...",['So loving case comes handy I want feminine a...,0,NaN
3,2648268,US,30355234,R9UD49V4TAN94,B00MOOF2EO,802920518,"iPhone 6 Case, SUPCASE Apple iPhone 6 Case 4.7...",Wireless,4,0,0,0,1,Good minimalist case.,A good minimalist bumper case that does the jo...,2015-01-27,moderate,A good minimalist bumper case job good price. ...,"['A', 'good', 'minimalist', 'bumper', 'case', ...",['A good minimalist bumper case job good price...,2,NaN
4,8912834,US,49597651,R1LQ6FA34TK3ZA,B000H84J3Y,177317905,TomTom ONE Carry Case and Strap (Discontinued ...,Wireless,4,0,0,0,0,Not bad but...,I got the TomTom XL-S for Christmas along with...,2007-12-26,moderate,I got TomTom XL-S Christmas along Carry case s...,"['I', 'got', 'TomTom', 'XL-S', 'Christmas', 'a...",['I got TomTom XL-S Christmas along Carry case...,2,NaN


In [16]:
train.shape , test.shape

((274534, 22), (68634, 22))

In [17]:
train.drop(train.tail(250000).index,inplace=True)
test.drop(test.tail(60000).index,inplace=True)

In [18]:
train.shape , test.shape

((24534, 22), (8634, 22))

In [19]:
encoded_data_train = tokenizer.batch_encode_plus(
    train.review_clean.astype(str),
    add_special_tokens = True,
    return_attention_mask = True,
    pad_to_max_length = True,
    max_length = 256,
    return_tensors = 'pt'
)

encoded_data_test = tokenizer.batch_encode_plus(
    test.review_clean.astype(str),
    add_special_tokens = True,
    return_attention_mask = True,
    pad_to_max_length = True,
    max_length = 256,
    return_tensors = 'pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(train.labels.values)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(test.labels.values)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [20]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

In [21]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = 3,
    output_attentions = False,
    output_hidden_states = False
)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [22]:
batch_size = 4 #32
dataloader_train = DataLoader(
    dataset_train,
    sampler = RandomSampler(dataset_train),
    batch_size = batch_size
)

dataloader_test = DataLoader(
    dataset_test,
    sampler = SequentialSampler(dataset_test),
    batch_size = 32
)

In [23]:
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5, #2e-5 to 5e-5
    eps = 1e-8
)


In [24]:
epochs = 2
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = len(dataloader_train)*epochs
)



In [25]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis = 1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average="weighted")    


In [26]:
def accuracy_per_class(preds, labels):
    labels_dict_inverse = {v: k for k,v in label_dict.items()}
    preds_flat = np.argmax(preds, axis = 1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat == label]
        y_true = labels_flat[labels_flat == label]
        print('Class: {}'.format(labels_dict_inverse[label]))
        print('Accuracy: {}\n'.format( len(y_preds[y_preds == label]) / len(y_true)) )

In [27]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [29]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_test):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    
    loss_train_total = 0
    progress_bar = tqdm(dataloader_train,
                        desc="Epoch {:1d}".format(epoch),
                        leave=False,
                        disable=False)
    
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids'       : batch[0],
            'attention_mask'  : batch[1],
            'labels'          : batch[2]
        }
    
        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix( {'training_loss': '{:3f}'.format(loss.item() / len(batch))} )

    torch.save(model.state_dict(), 'BERT_ft_epoch{}.model'.format(epoch))
    tqdm.write('\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total / len(dataloader_train)
    tqdm.write('Training loss: {}'.format(loss_train_avg))
    
    val_loss, predictions, true_vals = evaluate(dataloader_test)
    val_f1 = f1_score_func(predictions, true_vals)
    
    tqdm.write('Validation loss: {}'.format(val_loss))
    tqdm.write('f1 score (weighted): {}'.format(val_f1))


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/6134 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels= 3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
model.load_state_dict(
    torch.load('BERT_ft_epoch2.model',
              map_location=torch.device('cpu'))
)

<All keys matched successfully>

In [ ]:
_, predictions, true_vals = evaluate(dataloader_test)


  0%|          | 0/270 [00:00<?, ?it/s]

KeyboardInterrupt: ignored